In [1]:
!ollama pull llama2

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 8934d96d3f08... 100% ▕████████████████▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕████████████████▏ 7.0 KB                         
pulling 7c23fb36d801... 100% ▕████████████████▏ 4.8 KB                         
pulling 2e0493f67d0c... 100% ▕████████████████▏   59 B                         
pulling fa304d675061... 100% ▕████████████████▏   91 B                         
pulling 42ba7f8a01dd... 100% ▕████████████████▏  557 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling 8934d96d3f08... 100% ▕████████████████▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕████████████████▏ 7.0 KB 

In [2]:

from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [3]:
llm.invoke("how can langsmith help with testing?")

"\nLangSmith is a language model that can assist with testing in several ways:\n\n1. Automated Testing: LangSmith can be used to automate testing by generating test cases and executing them on the fly. This can save time and effort compared to manual testing, especially for complex systems or applications.\n2. Code Review: LangSmith can review code automatically and identify potential issues, such as syntax errors, inconsistent naming conventions, and security vulnerabilities. This can help developers write better code and catch problems early in the development process.\n3. Test Data Generation: LangSmith can generate test data automatically, including input values and scenarios that cover a wide range of possibilities. This can help ensure that the system or application is thoroughly tested and that all possible scenarios are covered.\n4. Test Case Design: LangSmith can assist in designing test cases by suggesting relevant inputs, expected outputs, and edge cases based on the code st

We can also guide its response with a prompt template. Prompt templates convert raw user input to better input to the LLM.

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [5]:
#chain them togather

chain = prompt | llm 

We can now invoke it and ask the same question. It still won't know the answer, but it should respond in a more proper tone for a technical writer!

In [6]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is an incredible tool for testing! With its advanced AI capabilities and natural language processing (NLP) algorithms, Langsmith can assist in various ways throughout the testing process. Here are some potential applications:\n\n1. Automated writing tests: Langsmith's NLP can analyze and understand technical documentation, including text, images, and other multimedia content. It can then generate automated tests based on this analysis, ensuring that your documentation is accurate, complete, and up-to-date.\n2. Error detection and reporting: Langsmith can scan through technical documentation and identify potential errors or inconsistencies, such as broken links, missing sections, or incorrect formatting. It can then flag these issues for review and correction, streamlining the testing process.\n3. Quality assessment: Langsmith's NLP can evaluate the quality of technical documentation based on factors like read

In [7]:
#The output of a ChatModel (and therefore, of this chain) is a message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()



In [8]:
#We can now add this to the previous chain:

chain = prompt | llm | output_parser

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I'm pleased to share that Langsmith can greatly assist with testing in several ways:\n\n1. Automated Testing: With Langsmith's AI capabilities, you can create automated tests for your software or applications. This helps ensure that the product functions as intended and identifies any bugs or issues before they become problems.\n2. Content Generation: Langsmith can generate test cases based on the product documentation, which can help ensure that all aspects of the product are thoroughly tested. This saves time and effort compared to creating test cases from scratch.\n3. Test Data Generation: Langsmith can also generate test data that mimics real-world scenarios, enabling you to test the product under various conditions. This helps identify potential issues before they impact users.\n4. Defect Reporting: In the event of a failure or bug, Langsmith can assist in documenting and reporting the defects. By using

# Retrieval Chain

First, we need to load the data that we want to index. To do this, we will use the WebBaseLoader. This requires installing BeautifulSoup:

In [10]:
!pip install beautifulsoup4

To properly answer the original question ("how can langsmith help with testing?"), we need to provide additional context to the LLM. We can do this via retrieval. Retrieval is useful when you have too much data to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in.

In this process, we will look up relevant documents from a Retriever and then pass them into the prompt. A Retriever can be backed by anything - a SQL table, the internet, etc - but in this instance we will populate a vector store and use that as a retriever. For more information on vectorstores, see this documentation.



In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

Next, we need to index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.

In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [13]:
#Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, FAISS, for simplicity's sake.

#First we need to install the required packages for that:

!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.4 MB/s eta 0:00:0000:0100:01


In [14]:
#Then we can build our index:

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to, we could run this ourselves by passing in documents directly:



In [16]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, Langsmith can help with testing by providing a visualization of test results. This means that Langsmith can provide a graphical representation of the data collected during testing, allowing users to easily understand and analyze the results. This can be particularly useful for complex systems or large datasets where it may be difficult to interpret raw data without some form of visualization. By providing a visual representation of the test results, Langsmith can help users identify trends, patterns, and insights that might otherwise be difficult to discern from raw data alone.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.



In [17]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

We can now invoke this chain. This returns a dictionary - the response from the LLM is in the answer key


In [18]:

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


Based on the provided context, LangSmith can help with testing in several ways:

1. Creating test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, that can be used to run tests on their LLM applications.
2. Annotating traces: LangSmith supports sending runs to annotation queues, which allow annotators to closely inspect interesting traces and annotate them with respect to different criteria. This helps catch regressions across important evaluation criteria.
3. Adding runs to a dataset: As the application progresses through the beta testing phase, it's essential to continue collecting data to refine and improve its performance. LangSmith enables users to add runs as examples to datasets, expanding test coverage on real-world scenarios.
4. Monitoring and A/B testing: LangSmith provides monitoring charts that allow users to track key metrics over time. They can also group traces from a single conversation together, making it ea

# Conversation Retrieval Chain

The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

We can still use the create_retrieval_chain function, but we need to change two things:

1. The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
2. he final LLM chain should likewise take the whole history into account

## Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query.



In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [20]:
#We can test this out by passing in an instance where the user asks a follow-up question.

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic responses. LangSmith allows you to attach feedback scores to logged traces (oftentime

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow-up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [23]:
#We can now test this out end-to-end:

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most i

We can see that this gives a coherent answer - we've successfully turned our retrieval chain into a chatbot!



# Agent

We've so far created examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

1. The retriever we just created. This will let it easily answer questions about LangSmith
2. A search tool. This will let it easily answer questions that require up-to-date information.


First, let's set up a tool for the retriever we just created: